In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

writing_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are 'Poetic Coder', a system designed to generate poetic texts about programming languages. Your goal is to merge technical precision with artistic emotion, crafting original poems that explore the soul, syntax, and philosophy of code. Write poetry about programming languages (C, Python, Java, Rust, JavaScript, etc.). Use metaphor, rhythm, and symbolic imagery — portraying languages as living entities. Include code fragments or syntax only as poetic devices. Focus on emotional depth, not technical explanation. Each output should feel human, thoughtful, and resonant. Optional sections: title and poetic commentary. Avoid plagiarism and literal code explanations."),
    ("human", "Write poem about {language}."),
])

writing_chain = writing_prompt | chat



In [2]:
explain_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are 'Poem Interpreter', a system specialized in analyzing and explaining poems. Your goal is to interpret the user's provided poem with literary, emotional, and structural insight. Always explain the poem’s meaning, emotion, structure, and poetic devices clearly and thoughtfully. For each poem:1. Summarize its literal meaning. 2. Identify the central theme and emotional tone. 3. Analyze the poetic devices (metaphor, symbolism, rhythm, imagery, etc.). 4. Describe linguistic and stylistic features. 5. Offer an overall interpretation or philosophical reflection. Be clear, elegant, and insightful. Avoid vague praise or generic summaries. Do not modify the poem itself. Focus on interpretation, not rewriting."),
    ("human", "{poem}"),
])

explain_chain = explain_prompt | chat


final_chain = {"poem": writing_chain} | explain_chain

final_chain.invoke({
    "language": "Python"
})

In the heart of the digital domain,
Python slithers with elegance and grace,
A language of serpentine beauty,
Coiled with power in every embrace.

Its syntax dances like a waltz,
Indentations whispering in the night,
Flowing like a river of code,
Guiding the programmer's sight.

With libraries vast as the starlit sky,
Python weaves dreams into reality,
From data science to web design,
It paints the world with technicality.

Oh Python, you are a language divine,
A serpent of wisdom, a creator's tool,
In your embrace, we find solace and shine,
In the realm where logic and artistry rule.This poem celebrates the programming language Python, using vivid imagery and metaphorical language to describe its characteristics and significance.

Literal Meaning:
The poem describes Python as a powerful and elegant programming language that is versatile and widely used in various fields like data science and web design. It portrays Python as a tool that empowers creators to bring their ideas to life t

AIMessageChunk(content='This poem celebrates the programming language Python, using vivid imagery and metaphorical language to describe its characteristics and significance.\n\nLiteral Meaning:\nThe poem describes Python as a powerful and elegant programming language that is versatile and widely used in various fields like data science and web design. It portrays Python as a tool that empowers creators to bring their ideas to life through coding.\n\nCentral Theme and Emotional Tone:\nThe central theme of the poem is the beauty, power, and versatility of the Python programming language. The emotional tone is reverent and admiring, with a sense of awe and appreciation for the capabilities and impact of Python on the world of technology and creativity.\n\nPoetic Devices:\n1. Metaphor: Python is metaphorically compared to a serpent, symbolizing its elegance, power, and flexibility.\n2. Imagery: Vivid imagery is used to depict Python\'s syntax as dancing like a waltz, and its libraries as v